# Tools

Some LLMs are capable of acknowledging when to use tools and how to use them. For example, the Ollama tools that are compatible with tool calling can be found in this web page:

https://ollama.com/search?c=tools

In this tutorial we are goind to examine how the models use (or not) tools, how tools can optimize their capabilities and how to properly design tools that have actual impact.

An LLM has been trained with data that reach up to a specific date in time and on a broad context of information. Tools enable them to get up-to-date information and receive specialized knowledge that was possibly not available, or not particularly succinct and salient to them during training.

In this tutorial we will use DuckDuckGo search engine to enable an LLM to get up-to-date information. We will also show how a Wikipedia search can enhance details of retrieved information, but also how the model can "misuse" the tools and actually get no useful information.

### Load model

First, let's start by loading a model that is compatible with tools.

In [2]:
from langchain_ollama import ChatOllama

# Initialize the ChatOllama model with the specified model name
model_name = 'qwen3-vl:4b'

# and initialize the ChatOllama instance
chat_model = ChatOllama(
    model=model_name,
    validate_model_on_init=True,
    temperature=0.7
)

## Search tool

I this section we will have a look at the DuckDuckGo search engine and what it returns. First let's try it out.

In this tool we can submit queries and get snippets (summaries + titles) fromt some top results (like the snippets we get from results when we search in Google).

In [3]:
from langchain_community.tools import DuckDuckGoSearchRun

ddg_search = DuckDuckGoSearchRun()

search_result = ddg_search.run('Who is the president of Venezuela?')

# This tool returns a string with the snippets of the search result
print(type(search_result))
print(search_result)

<class 'str'>
Venezuela DELCY RODRIGUEZ Who she is : Venezuela's interim president Portfolio: Maduro's former vice president , she's running the country under the oversight of the United States. Who is Nicolás Maduro? Nicolás Maduro, 63, is the president of Venezuela and has served as the oil-rich country's leader since 2013. He was born in the country's capital of Caracas. Rodríguez served as Maduro's vice president since 2018, overseeing much of Venezuela's oil-dependent economy and its feared intelligence service, and was next in the presidential line of succession. Key facts about Nicolás Maduro, the Venezuelan leader who U.S. President Donald Trump said had been captured by U.S. forces on Saturday. Venezuelan President Nicolas Maduro was sworn in for a third term in January 2025 following a 2024 election that was widely condemned by international observers and ...


## LangChain tool

To make a LangChain tool, we need to decorate a python function and let it return a value that the LLM can handle and integrate into its chat logic. Some things are important to notice here:

- The functions need to have a `@tool` decoration for LangChain to register them as such.
- The pydantic definition of functions are important for informing the LLM what the tool actually does. I.e., it is better to have tool function in this form `def foo(x: int) -> float:` rather than in this form `def foo(x):`.
- Comments in the tool function definition are important, because they allow the model to understand what the model does. Additionally, the comments need to be added in the *docstring* way so that LangChain can include them properly inside a structured schema that is injected in the LLM.

In [4]:
from langchain.tools import tool

In [5]:
@tool
def search(query: str) -> str:
    """Search for information on the internet using a search tool."""
    return ddg_search.run(query)

### With vs without the tool

Let's see how the model responds with and without the tool. For this, we will keep the version of the model without the tool and create a new instance that has the tool binded.

In [6]:
model_with_tool = chat_model.bind_tools([search])

In [7]:
result_no_tool = chat_model.invoke("Who is the President of Venezuela?")

In [8]:
print(result_no_tool.content)

As of my latest update (October 2023), **Nicolás Maduro** is the **President of Venezuela**, holding the office since 2013. Here's a clear breakdown of the situation:

1.  **Current Status:**
    *   Maduro was first elected President in 2013, following a controversial election. He was re-elected in 2018 and again in 2023.
    *   He is the **officially recognized President** by the Venezuelan government and the constitution of Venezuela.

2.  **Context & Controversy:**
    *   **Disputed Elections:** Maduro's 2013 and 2018 elections were widely disputed by opposition groups, international observers, and many foreign governments (including the U.S., EU, and Latin American nations). The opposition, led by Juan Guaidó, declared Maduro's victory invalid in 2014 and 2018, claiming he was a "dictator" and that the elections were "null and void."
    *   **International Recognition:** While Maduro's government claims legitimacy, **the majority of the international community does not recogniz

### Results without tool

At the time of writing this tutorial (January 2026) the model was trained with data up to July 2024 and the answer it gave was the following:

```
As of **July 2024**, the **de facto leader of Venezuela is Nicolás Maduro**, who has been the President since **2013**. However, the situation is highly contested due to the **political crisis and ongoing conflict** between Maduro's government and the opposition.

### Key Details:
1. **Official Status**:  
   - Maduro was **elected in 2013** and re-elected in **2018** (though the 2018 election was widely disputed).  
   - In **2023**, he won a controversial presidential election, which the opposition and many international observers deemed fraudulent.  
   - The **National Assembly** (controlled by Maduro's allies) declared a **"state of emergency"** in May 2023, suspending the presidency and forcing Maduro to step down temporarily. However, he **was re-elected in a disputed vote** later that year, and the National Assembly reinstated him.  

2. **International Recognition**:  
   - **United Nations, Brazil, Colombia, and other countries** recognize Maduro as the legitimate president.  
   - **The United States, Canada, the EU, and many Latin American nations** do not recognize his 2023 election, calling it a "sham" and supporting the opposition's claim of a "state of emergency."  
   - The **Venezuelan government** maintains Maduro is the rightful leader, while the **opposition (led by Juan Guaidó)** claims the 2023 election was invalid and that the presidency should be contested.  

3. **Current Situation**:  
   - Venezuela has been in a **prolonged political and economic crisis** since 2014, with severe shortages of food, medicine, and electricity.  
   - Maduro's government has **tightened control** over institutions, while the opposition has held protests and sought international support.  
   - In **April 2024**, the National Assembly **rejected Maduro's legitimacy**, but the Supreme Court later ruled he could stay in power until 2028.  

### Why the Confusion?  
- **Legitimacy**: Maduro's government claims he was elected by the Venezuelan people, but the opposition argues the 2023 election was rigged.  
- **Legal Ambiguity**: The National Assembly's actions in 2023 created a power vacuum, leading to the **2023 "state of emergency"** that suspended Maduro's presidency. However, he **was restored as president** after the National Assembly declared him the legitimate leader.  
- **International Context**: The U.S. and allies have **not recognized** Maduro's 2023 victory, while the UN and many countries **do**.  

### Conclusion:  
While **Nicolás Maduro** is the **officially recognized President of Venezuela** by the government and the UN, his position is **contested internationally** and marked by **ongoing civil unrest**. The situation remains fluid, with no clear resolution as of **July 2024**. For the most current updates, it's best to consult **Venezuelan government sources** or **reputable news outlets like Reuters or BBC**.  

*Note: This answer reflects the status as of July 2024. Political developments can change rapidly.*

```

### Using the tool

Making the LLM use the tool involves sending it a message with the question. In the message, and before the question, we have included a prompt to use the tool. This is not necessary but it increases the chances that the model will decide to use the tool.

After we pass the message to the tool, we check the message that came from the LLM and run the tools it may have decided to run - we will see later that we can bind the tool to several tools. We then have to run the tool, get the results and append it to the message so that we can give the entire information as a new message back to the tool for processing and final answer generation.

In [9]:
from langchain_core.messages import HumanMessage, ToolMessage

# We will use a HumanMessage and a ToolMessage for keeping better structure.

messages = [
    HumanMessage(
        content="You must use the search tool to answer this question: "
                "Who is the president of Venezuela?"
    )
]

# First turn: model probably calls the tool and this should appear in the ai_msg
ai_msg = model_with_tool.invoke(messages)
print("AI Message:", ai_msg)

# Append the AI message to the messages list
messages.append(ai_msg)

# Tool execution: check for tool calls
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        # Check the tool call structure
        print(tool_call)
        # Get the tool result
        tool_result = search.invoke(tool_call['args']['query'])
        # Append the tool result as a ToolMessage
        messages.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id'] # Access 'id' as a dictionary key as well
            )
        )

    # New human turn: append a HumanMessage asking to answer the question
    messages.append(
        HumanMessage(
            content="Using the tool results above, answer the question."
        )
    )

    final_response = model_with_tool.invoke(messages)
    print(final_response.content)
else:
    print("Model did not call tool:")
    # If no tool calls, print the AI message content
    print(ai_msg.content)

AI Message: content='' additional_kwargs={} response_metadata={'model': 'qwen3-vl:4b', 'created_at': '2026-01-22T04:59:47.969207173Z', 'done': True, 'done_reason': 'stop', 'total_duration': 4476057729, 'load_duration': 105774782, 'prompt_eval_count': 169, 'prompt_eval_duration': 140563575, 'eval_count': 243, 'eval_duration': 4146080932, 'logprobs': None, 'model_name': 'qwen3-vl:4b', 'model_provider': 'ollama'} id='lc_run--019be412-9c04-7e81-83c2-919c23538db1-0' tool_calls=[{'name': 'search', 'args': {'query': 'current president of Venezuela'}, 'id': 'eff0ea50-b6b6-430f-9147-e6743838d0f5', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 169, 'output_tokens': 243, 'total_tokens': 412}
{'name': 'search', 'args': {'query': 'current president of Venezuela'}, 'id': 'eff0ea50-b6b6-430f-9147-e6743838d0f5', 'type': 'tool_call'}
According to the most recent information provided in the tool results (from 2 days ago), **Delcy Rodríguez** is the current acting president 

### Tool result

We see above the the model decided to use the tool and pass as search query the term "current president of Venezuela". This shows that the model internally could resolve what information it needed to be retrieved based on the human input.

## WikipediaRetriever class

Like the search tool, the Wikipedia tool receives a search term and provides the Wikipedia pages, if any, that correspond to this result. It is not necessary that results will be produced for any query, while it is not necessary that the first result will be relevant to the search - the model may form a query to Wikipedia that is not proper.

First let's check the output of a `WikipediaRetriever` class object before we make a tool out of it.

In [9]:
from langchain_community.retrievers import WikipediaRetriever

In [10]:
retriever = WikipediaRetriever()

In [11]:
docs = retriever.invoke("Kasawari")

In [ ]:
print(type(docs))
print(type(docs[0]))
# Check the content of the first retrieved document
print(docs[0].page_content)

<class 'list'>
<class 'langchain_core.documents.base.Document'>
Cassowaries (Indonesian: kasuari; Biak: man suar 'bird strong'; Tok Pisin: muruk; Papuan: kasu weri 'horned head') are flightless birds of the genus Casuarius, in the order Casuariiformes. They are classified as ratites, flightless birds without a keel on their sternum bones. Cassowaries are native to the tropical forests of New Guinea (Western New Guinea and Papua New Guinea), the Moluccas (Seram and Aru Islands), and northeastern Australia.
Three cassowary species are extant. The most common, the southern cassowary, is the third-tallest and second-heaviest living bird, smaller only than the ostrich and emu. The other two species are the northern cassowary and the dwarf cassowary; the northern cassowary is the most recently discovered and the most threatened. A fourth, extinct, species is the pygmy cassowary.
Cassowaries are very wary of humans, but if provoked, they are capable of inflicting serious, even fatal, injuries

### Wikipedia tool

Now let's make the tool in a similar way as with the search tool, but we now return the first page of the `WikipediaRetriever` class object. We could try to append the results of the top `k` retrieved documents, but this would not necessarily increase the accuracy of the response, since the top `k` documents may include absolutely irrelevant information. You can try it out though.

In [34]:
@tool
def wikipedia_search(query: str) -> str:
    """Search for information on wikipedia using its search tool."""
    docs = retriever.invoke(query)
    return docs[0].page_content

In [35]:
model_with_tool = chat_model.bind_tools([search,wikipedia_search])

In [16]:
result_no_tool = chat_model.invoke("How many Kasawari species are there?")

In [17]:
print(result_no_tool.content)

The term **"Kasawari"** refers to a genus of fish in the family **Serranidae** (which includes groupers, sea basses, and other reef-dwelling fish). However, it's important to clarify the correct taxonomic name:  

- The genus is **Kasawaria**, not "Kasawari" (which is a common misspelling or misnomer).  
- **Kasawaria** is a monotypic genus, meaning it contains **only one species**:  
  **Kasawaria maris** (also known as the "Kasawari" or "Kasawari grouper").  

### Key Details:
- **Taxonomic status**:  
  - Kingdom: Animalia  
  - Phylum: Chordata  
  - Class: Actinopterygii (ray-finned fish)  
  - Order: Serraniformes  
  - Family: Serranidae  
  - Genus: *Kasawaria*  
  - Species: *Kasawaria maris* (validly described as a single species).  

- **Why confusion arises**:  
  - The name "Kasawari" is often used informally for *Kasawaria maris*, but the scientific genus is **Kasawaria**, not "Kasawari."  
  - Some sources may incorrectly use "Kasawari" as the genus name, but this is out

### Result without the tool

Without the tool the model answers that there is a single spices of Kasawari (which is not the correct answer):

```
The term **"Kasawari"** refers to a genus of fish in the family **Serranidae** (which includes groupers, sea basses, and other reef-dwelling fish). However, it's important to clarify the correct taxonomic name:  

- The genus is **Kasawaria**, not "Kasawari" (which is a common misspelling or misnomer).  
- **Kasawaria** is a monotypic genus, meaning it contains **only one species**:  
  **Kasawaria maris** (also known as the "Kasawari" or "Kasawari grouper").  

### Key Details:
- **Taxonomic status**:  
  - Kingdom: Animalia  
  - Phylum: Chordata  
  - Class: Actinopterygii (ray-finned fish)  
  - Order: Serraniformes  
  - Family: Serranidae  
  - Genus: *Kasawaria*  
  - Species: *Kasawaria maris* (validly described as a single species).  

- **Why confusion arises**:  
  - The name "Kasawari" is often used informally for *Kasawaria maris*, but the scientific genus is **Kasawaria**, not "Kasawari."  
  - Some sources may incorrectly use "Kasawari" as the genus name, but this is outdated. Modern taxonomy uses *Kasawaria*.  

- **Number of species**:  
  **1 species** (*Kasawaria maris*).  

### Additional Context:
- *Kasawaria maris* is a small, deep-water grouper found in the Indian and Pacific Oceans, typically at depths of 200–1000 meters. It is not a shark (sharks belong to the order Selachimorpha, while *Kasawaria* is a ray-finned fish).  
- The genus *Kasawaria* is **monotypic**—no other species are recognized in this genus.  

### Why This Matters:
If you encountered "Kasawari" in a non-scientific context (e.g., fishing or local names), it likely refers to *Kasawaria maris*. However, in formal taxonomy, the genus is **Kasawaria** with **1 species**.  

**Final Answer**: There is **1 species** in the genus *Kasawaria* (formerly mislabeled as "Kasawari").
```

In [37]:
from langchain_core.messages import HumanMessage, ToolMessage

# We will use a HumanMessage and a ToolMessage for keeping better structure.

messages = [
    HumanMessage(
        content="You can use any available tool to answer this question: "
                "How many spicies of Kasawari are there?"
    )
]

# First turn
ai_msg = model_with_tool.invoke(messages)
messages.append(ai_msg)

# Tool execution
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        print(tool_call)
        # Corrected: Access 'args' as a dictionary key and then 'query' within it
        tool_result = search.invoke(tool_call['args']['query'])
        messages.append(
            ToolMessage(
                content=str(tool_result),
                tool_call_id=tool_call['id'] # Access 'id' as a dictionary key as well
            )
        )

    # New human turn (required!)
    messages.append(
        HumanMessage(
            content="Using the tool results above, answer the question."
        )
    )

    final_response = model_with_tool.invoke(messages)
    print(final_response.content)
else:
    print("Model did not call tool:")
    print(ai_msg.content)

{'name': 'search', 'args': {'query': 'how many species of Kasawari are there'}, 'id': '11f93411-6230-411a-97ae-77fe481bd994', 'type': 'tool_call'}
There are three species of cassowaries, which are commonly referred to as Kasawari. The tool response confirms that "all three species of cassowaries are considered as Asia's largest bird since the extinction of the Arabian ostrich." Thus, the number of species is **three**.


### Different results

Depending on the randomness expressed by temperature, the model may go through different routes in answering the question. Below you can some indicative instances where the model was able to produce a proper query (asking about the number of kasawari spieces), get a proper answer and come up with a proper response. Here are some examples:

Used the `wikipedia_search` tool with a general query ("Kasawari") but got right results:
```
{'name': 'wikipedia_search', 'args': {'query': 'Kasawari'}, 'id': '18f1325f-db46-49d9-a337-6c3d26acfa8d', 'type': 'tool_call'}
There are three species of cassowaries, which are also known as "Kasawari" in Indonesian. The family Casuariidae includes these three species:  
1. Southern cassowary (*Casuarius casuarius*)  
2. Northern cassowary (*Casuarius unappendiculatus*)  
3. Dwarf cassowary (*Casuarius bennetti*)  

While the tool response mentions "four surviving members" (including the emu), the emu belongs to a different family (Dromaiidae) and is not part of the cassowary group. Thus, the correct count of *Kasawari* (cassowary) species is **three**.
```

Used the `search` tool with a general search term ("Kasawari") and got wrong results:
```
{'name': 'search', 'args': {'query': 'Kasawari'}, 'id': 'd6043885-5bab-474d-b30f-c85d7b1684dc', 'type': 'tool_call'}
Based on the provided search results, there is no specific information about the number of species of "Kasawari" (which appears to be a common name for a song, street, or cultural reference rather than a biological species). The search results do not mention any taxonomic classification, scientific species count, or biological context for "Kasawari." 

However, if "Kasawari" refers to the **Spotted Dove** (*Columba palumbus*), a bird species known by this name in some regions (e.g., the Philippines), it is a **single species**. But since the search results do not confirm this biological context, the answer cannot be definitively derived from the given data. 

**Conclusion**: The search results do not provide information about the number of *species* of "Kasawari." If this term refers to a biological entity, further clarification or a scientific database would be needed.
```

Used the `search` tool with a more specific search term ("how many species of Kasawari are there") and got correct results:
```
{'name': 'search', 'args': {'query': 'how many species of Kasawari are there'}, 'id': '11f93411-6230-411a-97ae-77fe481bd994', 'type': 'tool_call'}
There are three species of cassowaries, which are commonly referred to as Kasawari. The tool response confirms that "all three species of cassowaries are considered as Asia's largest bird since the extinction of the Arabian ostrich." Thus, the number of species is **three**.
```